In [1]:
import keras
import tensorflow as tf
import numpy as np

Using TensorFlow backend.


In [2]:
from Inputs import Inputs

In [3]:
import prep

In [4]:
sess = keras.backend.get_session()

In [5]:
batch_size = 10

In [6]:
train_images, train_labels, valid_images, valid_labels, n_classes = prep.prepare_data('/Users/fulop/Downloads/dogbreed/sample/')

In [7]:
train_inputs = Inputs(train_images, train_labels, n_classes, shuffle = True, batch_size = batch_size)
valid_inputs = Inputs(valid_images, valid_labels, n_classes, shuffle = True, batch_size = batch_size)
steps_per_epoch = int(np.floor(train_inputs.size / batch_size))

In [8]:
with tf.device('/cpu:0'):
            iterator_train = train_inputs.generate_iterator()
next_input, next_output = iterator_train.get_next()

with tf.device('/cpu:0'):
            iterator_valid = valid_inputs.generate_iterator()
next_valid_input, next_valid_output = iterator_valid.get_next()

In [9]:
model = keras.applications.resnet50.ResNet50(include_top=True, weights='imagenet', input_tensor=next_input, input_shape=None, pooling=None, classes=1000)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        input_1[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 112, 112, 64)  0           bn_conv1[0][0]                   
___________________________________________________________________________________________

In [10]:
print(train_labels)

[  3  50  60 119  30  89  99  69  73 101 102  27  23  94  13 111   9  55
  43  63  98   5   7 101  81  63 111  32 108  26  36  56  60 102  19  90
  24   3  52  58  48  42  84  40  73  16  31  73  86  70  69  93  58  53
  80  17 108 107   1  67   1  80  84  87  59  66 100  72 103  93  65  77
  94  71  67  87   4  63  89  69  75  20 115  53  54  25  52  56  38  90
  46 106  70 105 117  79  31   5  26  99 105  33 110  21   3  51 111   7
  74  32  14 117  81  14  93  10   1 119  39  50 104   1  36  45  51  17
  99  63  14  24  94  42 117  90  94   6  52  79  42 102  38   2  40  91
  88  73  77  35  80  91  22  74  91  82   0  57  27  34  21  77  10  99
 119  90  61  30   7  94  53 112  27  51  53  65  95 102   9 111  78 104
   2   6  89   8  38  83  18  47 107  65  67  24  70  43  83  64  42  96
  91  25]


In [11]:
model.layers.pop()
x = keras.layers.Input(tensor=next_input)
x = keras.layers.Dense(n_classes, name = "predictions")(model.layers[-1].output)
model = keras.models.Model(inputs=model.layers[0].input, outputs=x)

In [12]:
model.compile(optimizer=keras.optimizers.Adam(1e-3),
       loss='categorical_crossentropy', metrics=['accuracy'], target_tensors=[next_output])

In [13]:
sess.run(iterator_train.initializer)
sess.run(iterator_valid.initializer)
model.fit(steps_per_epoch = steps_per_epoch,validation_data=(next_valid_input, next_valid_output))

Epoch 1/1
19/20 [===========================>..] - ETA: 19s - loss: 5.2511 - acc: 0.0105

ValueError: Either the input data should have a defined shape, or steps should be specified.